# 1. TensorFlow Setup

In [ ]:
import tensorflow
print(tensorflow.__version__)

# 2. Importing libraries
As a first step, we import the necessary libraries. We use [Matplotlib](https://matplotlib.org/) for visualization purposes and [NumPy](https://numpy.org/) for computing on arrays.

In [ ]:
import os
import linecache
import math
from operator import itemgetter
import numpy as np
from numpy import zeros
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
plt.rcParams['font.family'] = 'serif'
plt.rcParams['font.serif'] = ['Times New Roman'] + plt.rcParams['font.serif']
plt.rcParams['font.size'] = '12'
from mpl_toolkits.mplot3d import Axes3D
import tensorflow as tf
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Reshape
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Convolution1D, MaxPooling1D
from tensorflow.keras.layers import Lambda, concatenate

# from tensorflow_graphics.math.interpolation.trilinear import interpolate

# 3 Pre-training with spine data

In [ ]:
Data = np.load('Data_spine.npy')
data_number = Data.shape[0]
# print(Data)
print('Number of data is:')
print(data_number, Data.shape[1], Data.shape[2])
print("///////////")
# print(len(Data[0]))
print("/////////////")
# print(len(Data[0][0]))
point_numbers = 5000

space_variable = 3 # 2 in 2D (x,y) and 3 in 3D (x,y,z)
cfd_variable = 3

input_data = zeros([data_number,point_numbers,space_variable],dtype='f')
output_data = zeros([data_number,point_numbers,cfd_variable],dtype='f')

for i in range(data_number):
    input_data[i,0:108,0] = Data[i,:,0] # x coordinate (m)
    input_data[i,0:108,1] = Data[i,:,1] # y coordinate (m)z
    input_data[i,0:108,2] = Data[i,:,2] # z coordinate (m)
    
    output_data[i,0:108,0] = Data[i,:,3] # u (m)
    output_data[i,0:108,1] = Data[i,:,4] # v (m)
    output_data[i,0:108,2] = Data[i,:,5] # p (m)
            
# print(input_data[0])
# print(output_data[0])
# print(input_data.shape[2])
# print(output_data.shape[1])
Root_Dir = os.path.abspath('')
visual_dir = os.path.join(Root_Dir, "predict_spine")
print(visual_dir)
if not os.path.exists(visual_dir):
    os.mkdir(visual_dir)


fout = open(os.path.join(visual_dir, "8_pred_spine.obj"), 'w')
for kk in range(point_numbers):
    fout.write(f"v {output_data[0][kk][0]} {output_data[0][kk][1]} {output_data[0][kk][2]}\n")

In [ ]:
u_min = np.min(output_data[:,:,0])
u_max = np.max(output_data[:,:,0])
v_min = np.min(output_data[:,:,1])
v_max = np.max(output_data[:,:,1])
p_min = np.min(output_data[:,:,2])
p_max = np.max(output_data[:,:,2])

output_data[:,:,0] = (output_data[:,:,0] - u_min)/(u_max - u_min)
output_data[:,:,1] = (output_data[:,:,1] - v_min)/(v_max - v_min)
output_data[:,:,2] = (output_data[:,:,2] - p_min)/(p_max - p_min)

In [ ]:
indices = np.random.permutation(input_data.shape[0])
training_idx, validation_idx, test_idx = indices[:int(0.8*data_number)], indices[int(0.8*data_number):int(0.9*data_number)], indices[int(0.9*data_number):]
input_training, input_validation, input_test = input_data[training_idx,:], input_data[validation_idx,:], input_data[test_idx,:]
output_training, output_validation, output_test = output_data[training_idx,:], output_data[validation_idx,:], output_data[test_idx,:]

In [ ]:
scaling = 0.25 #reasonable choices for scaling: 4.0, 2.0, 1.0, 0.25, 0.125

def exp_dim(global_feature, num_points):
    return tf.tile(global_feature, [1, num_points, 1])

PointNet_input = Input(shape=(point_numbers, space_variable))#point_numbers=10000, space_variable=3
#Shared MLP (64,64)
branch1 = Convolution1D(int(64*scaling),1,input_shape=(point_numbers,space_variable), activation='relu')(PointNet_input)
branch1 = BatchNormalization()(branch1)
branch1 = Convolution1D(int(64*scaling),1,input_shape=(point_numbers,space_variable), activation='relu')(branch1)
branch1 = BatchNormalization()(branch1)
Local_Feature = branch1
#Shared MLP (64,128,1024)
branch1 = Convolution1D(int(64*scaling),1,activation='relu')(branch1)
branch1 = BatchNormalization()(branch1)
branch1 = Convolution1D(int(128*scaling),1,activation='relu')(branch1)
branch1 = BatchNormalization()(branch1)
branch1 = Convolution1D(int(1024*scaling),1,activation='relu')(branch1)
branch1 = BatchNormalization()(branch1)
#Max function
Global_Feature = MaxPooling1D(pool_size=int(point_numbers*scaling))(branch1)
# Global_Feature = MaxPooling1D(pool_size=int(point_numbers*1))(branch1)
Global_Feature = Lambda(exp_dim, arguments={'num_points':int(scaling*point_numbers)})(Global_Feature)
branch2 = concatenate([Local_Feature, Global_Feature])
#Shared MLP (512,256,128)
branch2 = Convolution1D(int(512*scaling),1,activation='relu')(branch2)
branch2 = BatchNormalization()(branch2)
branch2 = Convolution1D(int(256*scaling),1,activation='relu')(branch2)
branch2 = BatchNormalization()(branch2)
branch2 = Convolution1D(int(128*scaling),1,activation='relu')(branch2)
branch2 = BatchNormalization()(branch2)
#Shared MLP (128, cfd_variable)
branch2 = Convolution1D(int(128*scaling),1,activation='relu')(branch2)
branch2 = BatchNormalization()(branch2)
PointNet_output = Convolution1D(cfd_variable,1,activation='sigmoid')(branch2) #Please note that we use the sigmoid activation function in the last layer.

In [ ]:
# from tensorflow.keras import backend as K
# def chamfer_distance_prev(y_true, y_pred):
#     """
#     Compute the Chamfer Distance between two point clouds
#     """
#     y_true = K.reshape(y_true, shape=(-1, 3))
#     y_pred = K.reshape(y_pred, shape=(-1, 3))

#     num_points_y_true = tf.shape(y_true)[0]
#     num_points_y_pred = tf.shape(y_pred)[0]

#     # Compute pairwise distance matrix
#     r_true = K.sum(y_true * y_true, axis=1)
#     r_true = K.reshape(r_true, [-1, 1])
#     r_pred = K.sum(y_pred * y_pred, axis=1)
#     r_pred = K.reshape(r_pred, [1, -1])

#     D = r_true - 2 * K.dot(y_true, K.transpose(y_pred)) + r_pred

#     # Compute the minimum distance for each point in y_true
#     min_distance_true = K.min(D, axis=1)

#     # Compute the minimum distance for each point in y_pred
#     min_distance_pred = K.min(D, axis=0)

#     return K.mean(min_distance_true) + K.mean(min_distance_pred)

In [ ]:
from tensorflow.keras import backend as K
def pairwise_l2_norm2_batch(y_true, y_pred):
    y_true = K.reshape(y_true, shape=(-1, 3))
    y_pred = K.reshape(y_pred, shape=(-1, 3))
    
    num_points_y_true = tf.shape(y_true)[0]
    num_points_y_pred = tf.shape(y_pred)[0]

    # Compute pairwise distance matrix
    r_true = K.sum(y_true * y_true, axis=1)
    r_true = K.reshape(r_true, [-1, 1])
    r_pred = K.sum(y_pred * y_pred, axis=1)
    r_pred = K.reshape(r_pred, [1, -1])

    D = r_true - 2 * K.dot(y_true, K.transpose(y_pred)) + r_pred
    return D

In [ ]:
from tensorflow.keras import backend as K

def chamfer_distance(y_true, y_pred):
    """
    Compute the Chamfer Distance between two point clouds
    """
    k_near = 8
    # calculate shape loss
    D = pairwise_l2_norm2_batch(y_true, y_pred)
    min_distance_true = K.min(D, axis=1)
    min_distance_pred = K.min(D, axis=0)
    shapeLoss = K.mean(min_distance_true) + K.mean(min_distance_pred)
    # calculate density loss
    D2 = pairwise_l2_norm2_batch(y_true, y_true)
    knndis = tf.nn.top_k(tf.negative(D), k=k_near)
    knndis2 = tf.nn.top_k(tf.negative(D2), k=k_near)
    densityLoss = tf.reduce_mean(tf.abs(knndis.values - knndis2.values))
    # Compute the minimum distance for each point in y_true
    # min_distance_true = K.min(D, axis=1)
    # data_loss = shapeLoss + densityLoss * 2
    # Compute the minimum distance for each point in y_pred
    # min_distance_pred = K.min(D, axis=0)
    data_loss = shapeLoss + densityLoss * 0.2
    # return K.mean(min_distance_true) + K.mean(min_distance_pred)
    return data_loss

In [ ]:
learning_rate = 0.0001
decaying_rate = 0.0
model = Model(inputs=PointNet_input,outputs=PointNet_output)

# model.compile(optimizers.Adam(lr=learning_rate, beta_1=0.9, beta_2=0.999, epsilon=0.000001, decay=decaying_rate)
#                    , loss='mean_squared_error', metrics=['mean_squared_error'])
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=learning_rate,
    decay_steps=10000,
    decay_rate=decaying_rate)
# optimizer = tf.keras.optimizers.Adam(learning_rate=lr_schedule)
optimizer = tf.keras.optimizers.Adam(learning_rate=lr_schedule, beta_1=0.9, beta_2=0.999, epsilon=0.00001)
# model.compile(optimizer, loss='mean_squared_error', metrics=['mean_squared_error'])
# model.compile(optimizer, loss=tf.keras.losses.MeanSquaredError(), metrics=[tf.keras.metrics.MeanSquaredError()])
checkpoint = ModelCheckpoint('fine_tuned_model.h5', save_best_only = True)
model.compile(optimizer, loss=chamfer_distance)


In [ ]:
batch = 4
epoch_number = 4000
results = model.fit(input_training,output_training,batch_size=batch,epochs=epoch_number,shuffle=True,verbose=1, validation_split=0.0, validation_data=(input_validation, output_validation), callbacks=[checkpoint])

# 4. Fine-tuning with trunk point cloud
For your convinient, we have already prepared data as a numpy array. The data for this specific test case is the spatial coordinates of the finite volume (or finite element) grids and the values of velocity and pressure fields on those grid points. The spatial coordinates are the input of PointNet and the velocity (in the *x* and *y* directions) and pressure fields are the output of PointNet. Here, our focus is on 3D cases.

In [ ]:
# Data = np.load('PointNetCFD/Data.npy')
Data = np.load('Data.npy')
# Data = np.load('_Data.npy')

# Data = 
data_number = Data.shape[0]
# print(Data)
print('Number of data is:')
print(data_number, Data.shape[1], Data.shape[2])
print("///////////")
# print(len(Data[0]))
print("/////////////")
# print(len(Data[0][0]))
point_numbers = 10000
space_variable = 3
cfd_variable = 3

input_data = zeros([data_number,point_numbers//2,space_variable],dtype='f')
output_data = zeros([data_number,point_numbers//2,cfd_variable],dtype='f')

for i in range(data_number):
    cnt = 0
    for kk in range(point_numbers):
        if kk % 2 == 0:
            # input_data[i,:,0] = Data[i,:,0] # x coordinate (m)
            # input_data[i,:,1] = Data[i,:,1] # y coordinate (m)
            # input_data[i,:,2] = Data[i,:,2] # z coordinate (m)

            # output_data[i,:,0] = Data[i,:,4] # u (m)
            # output_data[i,:,1] = Data[i,:,5] # v (m)
            # output_data[i,:,2] = Data[i,:,3] # p (m)
            input_data[i,cnt,0] = Data[i,kk,0] # x coordinate (m)
            input_data[i,cnt,1] = Data[i,kk,1] # y coordinate (m)
            input_data[i,cnt,2] = Data[i,kk,2] # z coordinate (m)

            output_data[i,cnt,0] = Data[i,kk,4] # u (m)
            output_data[i,cnt,1] = Data[i,kk,5] # v (m)
            output_data[i,cnt,2] = Data[i,kk,3] # p (m)

            cnt += 1
    
point_numbers = 10000 // 2
# print(input_data[0])
# print(output_data[0])
print(input_data.shape[2])
print(output_data.shape[1])
Root_Dir = os.path.abspath('')
visual_dir = os.path.join(Root_Dir, "predict")
print(visual_dir)
if not os.path.exists(visual_dir):
    os.mkdir(visual_dir)


fout = open(os.path.join(visual_dir, "8_pred.obj"), 'w')
for kk in range(point_numbers):
    fout.write(f"v {output_data[0][kk][0]} {output_data[0][kk][1]} {output_data[0][kk][2]}\n")
    

In [ ]:
u_min = np.min(output_data[:,:,0])
u_max = np.max(output_data[:,:,0])
v_min = np.min(output_data[:,:,1])
v_max = np.max(output_data[:,:,1])
p_min = np.min(output_data[:,:,2])
p_max = np.max(output_data[:,:,2])

output_data[:,:,0] = (output_data[:,:,0] - u_min)/(u_max - u_min)
output_data[:,:,1] = (output_data[:,:,1] - v_min)/(v_max - v_min)
output_data[:,:,2] = (output_data[:,:,2] - p_min)/(p_max - p_min)

In [ ]:
def plot2DPointCloud(x_coord,y_coord,z_coord,file_name):   
    plt.scatter(x_coord,y_coord,z_coord,s=2.5)
    plt.xlabel('x (m)')
    plt.ylabel('y (m)')
    plt.zlabel('z (m)')
    x_upper = np.max(x_coord) + 1
    x_lower = np.min(x_coord) - 1
    y_upper = np.max(y_coord) + 1
    y_lower = np.min(y_coord) - 1
    z_upper = np.max(z_coord) + 1
    z_lower = np.min(z_coord) - 1
    plt.xlim([x_lower, x_upper])
    plt.ylim([y_lower, y_upper])
    plt.zlim([z_lower, z_upper])
    plt.gca().set_aspect('equal', adjustable='box')
    plt.savefig(f'{file_name}.png', dpi=300)
    #plt.savefig(file_name+'.eps') #You can use this line for saving figures in EPS format   
    plt.clf()
    #plt.show()

def plotSolution(x_coord,y_coord,z_coord,solution,file_name,title):
    plt.scatter(x_coord,y_coord,z_coord,s=2.5,c=solution,cmap='jet')
    plt.title(title)
    plt.xlabel('x (m)')
    plt.ylabel('y (m)')
    plt.zlabel('z (m)')
    x_upper = np.max(x_coord) + 1
    x_lower = np.min(x_coord) - 1
    y_upper = np.max(y_coord) + 1
    y_lower = np.min(y_coord) - 1
    z_upper = np.max(z_coord) + 1
    z_lower = np.min(z_coord) - 1
    plt.xlim([x_lower, x_upper])
    plt.ylim([y_lower, y_upper])
    plt.zlim([z_lower, z_upper])
    plt.gca().set_aspect('equal', adjustable='box')
    cbar= plt.colorbar()
    #cbar.set_label(label, labelpad=+1)
    plt.savefig(f'{file_name}.png', dpi=300)
    #plt.savefig(file_name+'.eps') #You can use this line for saving figures in EPS format
    plt.clf()
    #plt.show()
    

number = 1 #It should be less than number of data ('data_number')
plot2DPointCloud(input_data[number,:,0],input_data[number,:,1],input_data[number,:,2],'PointCloud')
plotSolution(input_data[number,:,0],input_data[number,:,1],output_data[number,:,0],'u_velocity','normalized u (x-velocity component)')
plotSolution(input_data[number,:,0],input_data[number,:,1],output_data[number,:,1],'v_velocity','normalized v (y-velocity component)')
plotSolution(input_data[number,:,0],input_data[number,:,1],output_data[number,:,2],'pressure','normalized pressure')

In [ ]:
indices = np.random.permutation(input_data.shape[0])
training_idx, validation_idx, test_idx = indices[:int(0.8*data_number)], indices[int(0.8*data_number):int(0.9*data_number)], indices[int(0.9*data_number):]
input_training, input_validation, input_test = input_data[training_idx,:], input_data[validation_idx,:], input_data[test_idx,:]
output_training, output_validation, output_test = output_data[training_idx,:], output_data[validation_idx,:], output_data[test_idx,:]

In [ ]:

scaling = 0.25 #reasonable choices for scaling: 4.0, 2.0, 1.0, 0.25, 0.125

def exp_dim(global_feature, num_points):
    return tf.tile(global_feature, [1, num_points, 1])

PointNet_input = Input(shape=(point_numbers, space_variable))#point_numbers=10000, space_variable=3
#Shared MLP (64,64)
branch1 = Convolution1D(int(64*scaling),1,input_shape=(point_numbers,space_variable), activation='relu')(PointNet_input)
branch1 = BatchNormalization()(branch1)
branch1 = Convolution1D(int(64*scaling),1,input_shape=(point_numbers,space_variable), activation='relu')(branch1)
branch1 = BatchNormalization()(branch1)
Local_Feature = branch1
#Shared MLP (64,128,1024)
branch1 = Convolution1D(int(64*scaling),1,activation='relu')(branch1)
branch1 = BatchNormalization()(branch1)
branch1 = Convolution1D(int(128*scaling),1,activation='relu')(branch1)
branch1 = BatchNormalization()(branch1)
branch1 = Convolution1D(int(1024*scaling),1,activation='relu')(branch1)
branch1 = BatchNormalization()(branch1)
#Max function
# Global_Feature = MaxPooling1D(pool_size=int(point_numbers*scaling))(branch1)
Global_Feature = MaxPooling1D(pool_size=int(point_numbers*scaling))(branch1)
Global_Feature = Lambda(exp_dim, arguments={'num_points':int(point_numbers*scaling)})(Global_Feature)
branch2 = concatenate([Local_Feature, Global_Feature])
#Shared MLP (512,256,128)
branch2 = Convolution1D(int(512*scaling),1,activation='relu')(branch2)
branch2 = BatchNormalization()(branch2)
branch2 = Convolution1D(int(256*scaling),1,activation='relu')(branch2)
branch2 = BatchNormalization()(branch2)
branch2 = Convolution1D(int(128*scaling),1,activation='relu')(branch2)
branch2 = BatchNormalization()(branch2)
#Shared MLP (128, cfd_variable)
branch2 = Convolution1D(int(128*scaling),1,activation='relu')(branch2)
branch2 = BatchNormalization()(branch2)
PointNet_output = Convolution1D(cfd_variable,1,activation='sigmoid')(branch2) #Please note that we use the sigmoid activation function in the last layer.

In [ ]:
from tensorflow.keras import backend as K
def pairwise_l2_norm2_batch(y_true, y_pred):
    y_true = K.reshape(y_true, shape=(-1, 3))
    y_pred = K.reshape(y_pred, shape=(-1, 3))
    
    num_points_y_true = tf.shape(y_true)[0]
    num_points_y_pred = tf.shape(y_pred)[0]

    # Compute pairwise distance matrix
    r_true = K.sum(y_true * y_true, axis=1)
    r_true = K.reshape(r_true, [-1, 1])
    r_pred = K.sum(y_pred * y_pred, axis=1)
    r_pred = K.reshape(r_pred, [1, -1])

    D = r_true - 2 * K.dot(y_true, K.transpose(y_pred)) + r_pred
    return D

In [ ]:
from tensorflow.keras import backend as K

def chamfer_distance(y_true, y_pred):
    """
    Compute the Chamfer Distance between two point clouds
    """
    k_near = 8
    # calculate shape loss
    D = pairwise_l2_norm2_batch(y_true, y_pred)
    min_distance_true = K.min(D, axis=1)
    min_distance_pred = K.min(D, axis=0)
    shapeLoss = K.mean(min_distance_true) + K.mean(min_distance_pred)
    # calculate density loss
    D2 = pairwise_l2_norm2_batch(y_true, y_true)
    knndis = tf.nn.top_k(tf.negative(D), k=k_near)
    knndis2 = tf.nn.top_k(tf.negative(D2), k=k_near)
    densityLoss = tf.reduce_mean(tf.abs(knndis.values - knndis2.values))
    # Compute the minimum distance for each point in y_true
    # min_distance_true = K.min(D, axis=1)
    # data_loss = shapeLoss + densityLoss * 2
    # Compute the minimum distance for each point in y_pred
    # min_distance_pred = K.min(D, axis=0)
    data_loss = shapeLoss + densityLoss * 0.2
    # return K.mean(min_distance_true) + K.mean(min_distance_pred)
    return data_loss

In [ ]:
learning_rate = 0.0001
decaying_rate = 0.0
model = Model(inputs=PointNet_input,outputs=PointNet_output)

# model.compile(optimizers.Adam(lr=learning_rate, beta_1=0.9, beta_2=0.999, epsilon=0.000001, decay=decaying_rate)
#                    , loss='mean_squared_error', metrics=['mean_squared_error'])
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=learning_rate,
    decay_steps=10000,
    decay_rate=decaying_rate)
# optimizer = tf.keras.optimizers.Adam(learning_rate=lr_schedule)
optimizer = tf.keras.optimizers.Adam(learning_rate=lr_schedule, beta_1=0.9, beta_2=0.999, epsilon=0.00001)
# model.compile(optimizer, loss='mean_squared_error', metrics=['mean_squared_error'])
# model.compile(optimizer, loss=tf.keras.losses.MeanSquaredError(), metrics=[tf.keras.metrics.MeanSquaredError()])

checkpoint = ModelCheckpoint('fine_tuned_model.h5', save_best_only = True)
model.compile(optimizer, loss=chamfer_distance)


In [ ]:
batch = 4
epoch_number = 500

# pretrained_weights = tf.keras.models.load_model('fine_tuned_model.h5', custom_objects={'chamfer_distance':chamfer_distance})
# model.set_weights(pretrained_weights.get_weights())
results = model.fit(input_training,output_training,batch_size=batch,epochs=epoch_number,shuffle=True,verbose=1, validation_split=0.0, validation_data=(input_validation, output_validation), callbacks=[checkpoint])

In [ ]:
import scipy
from scipy.spatial import KDTree
def calculate_curvature(point_cloud, radius):
    # Create KDTree for efficient nearest neighbor search
    kdtree = KDTree(point_cloud)

    curvatures = []

    for point in point_cloud:
        # Find nearest neighbors within the given radius
        _, indices = kdtree.query(point, k=radius)

        # Extract the neighborhood points
        neighborhood = point_cloud[indices]

        # Calculate the covariance matrix
        covariance_matrix = np.cov(neighborhood.T)

        # Calculate eigenvalues of the covariance matrix
        eigenvalues, _ = np.linalg.eig(covariance_matrix)

        # Sort eigenvalues in ascending order
        eigenvalues.sort()

        # Calculate curvature using the formula
        curvature = (2 * eigenvalues[0]) / (eigenvalues[0] + eigenvalues[1])

        curvatures.append(curvature)

    return curvatures
for i in range(output_test.shape[0]):
    print(calculate_curvature(output_test[i], 8))

In [ ]:
def curvature(points):
    # Calculate the curvature of a 3D point cloud using TensorFlow
    print(points.shape)
    x, y, z = tf.split(points, num_or_size_splits=3, axis=2)
    print(x.shape)
    print(tf.concat([x, y, z], axis=2).shape)
    dx_dt, dy_dt, dz_dt = tf.image.image_gradients(tf.reshape(tf.concat([x, y, z], axis=2), (5, 5000, 3, 1)))
    ds_dt = tf.sqrt(dx_dt**2 + dy_dt**2 + dz_dt**2)
    d2x_ds2, _ = tf.image.image_gradients(dx_dt / ds_dt)
    _, d2y_ds2 = tf.image.image_gradients(dy_dt / ds_dt)
    _, _, d2z_ds2 = tf.image.image_gradients(dz_dt / ds_dt)
    curvature = (d2x_ds2 + d2y_ds2 + d2z_ds2) / ds_dt
    return curvature
print(curvature(output_test))

In [ ]:
import itertools
import sys

test_set_number = test_idx.size
print(test_set_number)
sample_point_cloud = zeros([1,point_numbers,space_variable],dtype='f')
truth_point_cloud = zeros([point_numbers,cfd_variable],dtype='f')
Root_Dir = os.path.abspath('')
visual_dir = os.path.join(Root_Dir, "predict")
print(visual_dir)
if not os.path.exists(visual_dir):
    os.mkdir(visual_dir)


# for j in range(point_numbers):
#         for i in range(space_variable):
print(input_data.shape[0])
# exit()
for s in range(input_data.shape[0]):
    for j, i in itertools.product(range(point_numbers), range(space_variable)):
        # sample_point_cloud[0][j][i] = input_test[s][j][i]
        sample_point_cloud[0][j][i] = input_data[s][j][i]

    prediction = model.predict(sample_point_cloud, batch_size=None, verbose=0)
    ########### calc point separation rate #########
    # dist = pairwise_l2_norm2_batch(prediction, output_test[s])
    # Unnormalized
    prediction[0,:,0] = prediction[0,:,0]*(u_max - u_min) + u_min
    prediction[0,:,1] = prediction[0,:,1]*(v_max - v_min) + v_min
    prediction[0,:,2] = prediction[0,:,2]*(p_max - p_min) + p_min
    # print(prediction.shape)
    ###################show obj##################
    fout = open(os.path.join(visual_dir, f"{s}_pred.obj"), 'w')
    for kk in range(point_numbers):
        # print(kk)
        fout.write(f"v {prediction[0,:,0][kk]} {prediction[0,:,1][kk]} {prediction[0,:,2][kk]}\n")
    
    #############################################
    # output_test[s,:,0] = output_test[s,:,0]*(u_max - u_min) + u_min
    # output_test[s,:,1] = output_test[s,:,1]*(v_max - v_min) + v_min
    # output_test[s,:,2] = output_test[s,:,2]*(p_max - p_min) + p_min
# exit()
# print(tf.reduce_mean(calculate_curvature(prediction[0], 8)) - tf.reduce_mean(calculate_curvature(output_test[0], 8)))
# exit()
D = pairwise_l2_norm2_batch(prediction, output_test)
min_distance_true = K.min(D, axis=1)
min_distance_pred = K.min(D, axis=0)
shapeLoss = K.mean(min_distance_true) + K.mean(min_distance_pred)
sepRate = K.mean(min_distance_true) / (1 - 0)
print(chamfer_distance(prediction, output_test), shapeLoss, sepRate)
# curvature1 = curvature(prediction[0])
# curvature2 = curvature(output_test[0])
# curvature_diff = tf.reduce_mean(tf.abs(curvature1 - curvature2)).numpy()
# print("Curvature difference:", curvature_diff)